# Generate SPOCK training data when integrating to one secular time scale

In [1]:
import spock
import random
import numpy as np
import rebound
import pandas as pd
from spock import simsetup
from spock import FeatureClassifier

load dataset

In [2]:
#specify the data path
#We will be using cleaned data generated from the original spock initial conditions data, filtered according to https://github.com/Ethadhani/SPOCKcleanData.git
datapath = '../../cleanData/csvs/resonant/'
initial = pd.read_csv(datapath+'clean_initial_conditions.csv')
labels = pd.read_csv(datapath+'clean_labels.csv')
#drop junk column
initial = initial.drop('Unnamed: 0', axis = 1)
#merge labels and initial conditions based on runstring
Initialdataset = initial.set_index('runstring').join(labels.set_index('runstring'))
Initialdataset = Initialdataset.drop('Unnamed: 0', axis = 1)

We can establish a function that, given a list of initial conditions, will return a rebound simulation

In [3]:
def get_sim(row, dataset):
    '''Given a row number, and a data sheet containing initial conditions, returns a corresponding simulation
    
        Arguments:
            row: what row the simulation you would like to create is on
                format of row is in order: 
                [index, 'p0m', 'p0x', 'p0y', 'p0z', 'p0vx', 'p0vy', 'p0vz', 'p1m', 'p1x', 'p1y',
                'p1z', 'p1vx', 'p1vy', 'p1vz', 'p2m', 'p2x', 'p2y', 'p2z', 'p2vx',
                'p2vy', 'p2vz', 'p3m', 'p3x', 'p3y', 'p3z', 'p3vx', 'p3vy', 'p3vz']

            dataset: what dataset contains your initial conditions

        return: returns a rebound simulation with the specified initial conditions'''
    try:
        data = dataset.loc[row]
        sim = rebound.Simulation()
        sim.G=4*np.pi**2
        sim.add(m=data['p0m'], x=data['p0x'], y=data['p0y'], z=data['p0z'], vx=data['p0vx'], vy=data['p0vy'], vz=data['p0vz'])
        sim.add(m=data['p1m'], x=data['p1x'], y=data['p1y'], z=data['p1z'], vx=data['p1vx'], vy=data['p1vy'], vz=data['p1vz'])
        sim.add(m=data['p2m'], x=data['p2x'], y=data['p2y'], z=data['p2z'], vx=data['p2vx'], vy=data['p2vy'], vz=data['p2vz'])
        sim.add(m=data['p3m'], x=data['p3x'], y=data['p3y'], z=data['p3z'], vx=data['p3vx'], vy=data['p3vy'], vz=data['p3vz'])
        return sim
    except:
        print("Error reading initial condition {0}".format(row))
        return None

We can now generate the set of system row indices

In [4]:
#generates the indexes of the systems
systemNum = range(Initialdataset.shape[0])

We can note the column names and import the different feature generators

In [5]:
col = ['EMcrossnear', 'EMfracstdnear', 'EPstdnear', 'MMRstrengthnear', 'EMcrossfar', 'EMfracstdfar', 'EPstdfar', 'MMRstrengthfar', 'MEGNO', 'MEGNOstd','Tmax', 'InitialStable']

In [6]:
spock = FeatureClassifier()

We can then establish some helper functions that will allow us to map the spock.generate_feature function to the different systems by mapping to different row numbers and generating the correct simulation

In [7]:
def getList(features):
    '''Helper function which isolates the data list from the generate_features return'''
    return list(features[0][0].values())+[features[1]]

In [8]:
def getFeat(num):
    '''when given a index of a row, loads initial conditions and returns the spock generated features'''
    #gets features based on index num
    sim = get_sim(num,initial)
    return spock.generate_features(sim, Tmax = True) #currently set to intigrate to 1Tsec

In [9]:
rebound.__version__

'4.3.2'

We can now map getFeat to the different rows of the Initial df, this will create each simulation and generate the spock features.

In [10]:
%%time
import sys
from multiprocessing import Pool
sys.path.append('spockUpdate/train_models')
#import is required to get around jupyter notebook bug with multiprocessing
if __name__ == "__main__":
    with Pool() as pool:
        features = pool.map(getFeat,systemNum)
        pool.close()
        pool.join()
#formats the data correctly

CPU times: user 3.07 s, sys: 597 ms, total: 3.67 s
Wall time: 27min 40s


In [11]:
formattedFeat = pd.DataFrame(np.array(list(map(getList,features))), columns = col)

We can then join the generated features with the corresponding labels

In [12]:
dataset = pd.DataFrame.join(formattedFeat,labels)

In [13]:
dataset

,EMcrossnear,EMfracstdnear,EPstdnear,MMRstrengthnear,EMcrossfar,EMfracstdfar,EPstdfar,MMRstrengthfar,MEGNO,MEGNOstd,Tmax,InitialStable,Unnamed: 0,runstring,instability_time,shadow_instability_time,Stable
0,0.060234,0.029458,0.001995,0.493311,0.504063,0.002902,0.000816,NaN,1.998619,0.003190,28431.468891,1.0,0,0000000.bin,1.545872e+06,3.063700e+06,False
1,0.080547,0.016559,0.000112,0.452015,0.240504,0.006736,0.001621,0.008674,2.001688,0.004966,3904.312508,1.0,1,0000001.bin,9.990000e+08,9.990000e+08,True
2,0.129660,0.028840,0.003182,1.000007,1.001981,0.001402,0.003742,0.010417,1.995051,0.003312,70073.708587,1.0,2,0000002.bin,9.990000e+08,9.990000e+08,True
3,0.406112,0.036450,0.002390,0.316676,0.427768,0.036189,0.008580,0.012898,2.002498,0.000335,18314.546267,1.0,3,0000003.bin,2.287671e+06,8.392234e+06,False
4,0.059897,0.028003,0.001623,0.334384,0.257596,0.053194,0.001287,0.034789,2.029737,0.013224,4110.218384,1.0,4,0000004.bin,9.668931e+05,3.380350e+05,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102492,0.089252,NaN,NaN,NaN,0.310694,NaN,NaN,NaN,NaN,NaN,23579.726004,0.0,113537,9044761.bin,6.303165e+04,6.470086e+04,False
102493,0.082222,0.039520,0.006137,0.680177,0.664222,0.030981,0.001015,NaN,1.999417,0.004993,4127.660202,1.0,113538,9045377.bin,6.990387e+05,8.267916e+05,False
102494,0.131799,0.022001,0.000148,0.875319,0.366664,0.060675,0.016560,0.005155,2.079854,0.076477,1130.408347,1.0,113540,9045380.bin,1.193822e+07,3.363291e+07,False
102495,0.209454,0.036894,0.005252,1.492760,0.395073,0.102268,0.022792,0.029797,1.875272,0.066749,4234.380683,1.0,113541,9045382.bin,2.064407e+08,4.316851e+07,False


We can then save the new training data spreadsheet.

In [14]:
dataset.to_csv(datapath+'Int1Tsec.csv')